# Problema 3 - Análisis Datos Youtube - Archivo .py

1. Descargar un archivo .zip mediante código del siguiente url (https://netsg.cs.sfu.ca/youtubedata/) (recomiendo descargar el archivo 0333.zip que es menos pesado)
2. Descomprimir los datos en una carpeta que genere y leer mediante pandas alguno de los archivos en esta. (observar que no es necesario en un primer momento leer los datos con un nombre de columna especifico)

    - Los nombres de columna pueden ser puestos posteriormente
    - El separador de columna es <code>\t</code>
    - Se colocan los nombres de columnas y descripción asociada para su intermetación. Ejemplo columna1 sera VideoID ... 
    

3. Procesar los datos según: 
    - Nos quedaremos con las columnas: VideoID, edad, catgoria, views, rate.
    - Realizar un filtrado básico a los datos. Ejemplo solo seleccionar cierto grupo de categorias

4. Procesamiento en Mongo Db
    - Exportar los datos a mongo DB 
    - Crear 2 graficos con los datos 
    - Compartir link donde encontrar los datos 




| Nombre de la Columna | Descripción                                                                                                 |
|----------------------|-------------------------------------------------------------------------------------------------------------|
| `video ID`           | Una cadena de 11 dígitos, la cual es única                                                                |
| `uploader`           | Una cadena con el nombre de usuario del cargador del video                                                  |
| `age`                | Un número entero que representa los días transcurridos desde la fecha en que se subió el video hasta el 15 de febrero de 2007 (fecha de creación de YouTube) |
| `category`           | Una cadena que indica la categoría del video elegida por el cargador                                       |
| `length`             | Un número entero que representa la duración del video en minutos                                            |
| `views`              | Un número entero que representa el número de visualizaciones del video                                      |
| `rate`               | Un número flotante que indica la calificación del video                                                      |
| `ratings`            | Un número entero que representa el número de calificaciones recibidas por el video                          |
| `comments`           | Un número entero que indica el número de comentarios en el video                                            |
| `related IDs`        | Hasta 20 cadenas de texto con los IDs de videos relacionados                                                |


In [11]:
import requests

url = 'https://netsg.cs.sfu.ca/youtubedata/0515.zip'
local_filename = '0515.zip'

# Descargar el archivo
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

import zipfile
import os

# Descomprimir el archivo
with zipfile.ZipFile(local_filename, 'r') as zip_ref:
    zip_ref.extractall('youtubedata')

# Listar archivos en la carpeta descomprimida
extracted_files = os.listdir('youtubedata')
print("Archivos extraídos:", extracted_files)

import pandas as pd

# Suponemos que el archivo de datos relevante es el primero de la lista
data_file = os.path.join('youtubedata', extracted_files[0])

# Leer el archivo .txt
with open(data_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Dividir las líneas en columnas usando '\t' como separador
data = [line.strip().split('\t') for line in lines]

# Convertir a DataFrame y asignar nombres a las columnas
df = pd.DataFrame(data, columns=['VideoID', 'Uploader', 'Age', 'Category', 'Length', 'Views', 'Rate', 'Ratings', 'Comments', 'RelatedIDs'])
# Seleccionar las columnas deseadas
df_filtered = df[['VideoID', 'Age', 'Category', 'Views', 'Rate']]

# Filtrar por categorías específicas (por ejemplo, 'Music' y 'Sports')
filtered_categories = ['Music', 'Sports']
df_filtered = df_filtered[df_filtered['Category'].isin(filtered_categories)]

from pymongo import MongoClient
import matplotlib.pyplot as plt

# Conectar a MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['youtube_data']
collection = db['videos']

# Convertir el DataFrame a un diccionario y cargarlo a MongoDB
data_dict = df_filtered.to_dict("records")
collection.insert_many(data_dict)

# Crear gráficos
# Gráfico 1: Distribución de categorías
category_counts = df_filtered['Category'].value_counts()
category_counts.plot(kind='bar')
plt.title('Distribution of Categories')
plt.xlabel('Category')
plt.ylabel('Count')
plt.savefig('category_distribution.png')
plt.show()

# Gráfico 2: Relación entre Views y Rate
plt.scatter(df_filtered['Views'], df_filtered['Rate'])
plt.title('Views vs. Rate')
plt.xlabel('Views')
plt.ylabel('Rate')
plt.savefig('views_vs_rate.png')
plt.show()



Archivos extraídos: ['0515']


IsADirectoryError: [Errno 21] Is a directory: 'youtubedata/0515'